# 個体間関係のラベル付け

## おまじないパート

In [1]:
import sys
from pathlib import Path

# 現在のノートブックのパスを取得
notebook_dir = Path().resolve() 

# プロジェクトルートディレクトリ（notebooksディレクトリの親ディレクトリ）を取得
# プロジェクトルートは3階層上です
project_root = notebook_dir.parent.parent.parent

# プロジェクトルートをPythonの検索パスに追加
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
from groom import GroomAim2, \
    save_to_parquet, load_from_parquet, \
        save_model, load_model

In [3]:
self = GroomAim2()

## data準備

### raw_nose_face_dfの読み込み

In [4]:
raw_nose_face_df = load_from_parquet(
    "../../../data/03_aim_1_data/corrected_nose_face_df.parquet"
)

# 不要な列を削除
raw_nose_face_df = raw_nose_face_df.drop(
    columns=['t0_flag', 
             'delta_face','delta_nose',
             'shade', 'shade_condition', 
             'is_role_swapped', 'has_Sg',], 
    errors='ignore'
)

--- 復元完了 ---
復元されたDataFrameのshape: (80367, 21)


In [5]:
print(raw_nose_face_df.columns)

Index(['delta_time', 'sampling_id', 'datetime', 'name', 'id', 'sex',
       'behavior', 'to', 'from', 'kin', 'corrected_delta_nose',
       'corrected_delta_face', 'nose-face', 'corrected_nose-face'],
      dtype='object')


In [6]:
# 'from'列と'to'列の両方に値がある（NaNでない）行を抽出
filtered_df = raw_nose_face_df.dropna(subset=['from', 'to'])

# 列を指定して表示
print(len(filtered_df[['sampling_id', 'name', 'from', 'to']]))

0


オスは相手方としてしか出現しない

In [7]:
print(len(raw_nose_face_df[raw_nose_face_df['name'] == 'Kobu']))
print(len(raw_nose_face_df[raw_nose_face_df['name'] == 'Nishin']))
print(len(raw_nose_face_df[raw_nose_face_df['name'] == 'Gure']))

0
0
0


### grooming_dfとgroomed_dfの作成

#### まずは分割

In [8]:
raw_grooming_df = raw_nose_face_df[raw_nose_face_df['behavior']=='grooming'].copy()
raw_groomed_df = raw_nose_face_df[raw_nose_face_df['behavior']=='groomed'].copy()

### fromおよびto列の補完

In [9]:
grooming_df = self.fix_grooming_records(
    raw_grooming_df,
    'grooming'
)

--- Mode: grooming (Counterpart column: to) ---
記録ミスの可能性がある sampling_id: [8, 73]
入れ替え処理が完了しました。
from 列を name 列の値で補完しました。

[処理結果の確認(代表各1行)]
      sampling_id      name      from        to
602             3     Botan     Botan   Komatsu
1204            5     Botan     Botan   Komatsu
1806            8     Botan     Botan   Komatsu
2408           10     Botan     Botan  Botaninf
3311           15     Kanna     Kanna      Yuna
4214           20  Tsutsuji  Tsutsuji      Gure
4515           22  Tsutsuji  Tsutsuji      Kobu
4816           24  Tsutsuji  Tsutsuji      Kobu
5117           26  Tsutsuji  Tsutsuji     Tsuwa
5719           28  Tsutsuji  Tsutsuji     Tsuwa


In [10]:
groomed_df = self.fix_grooming_records(
    raw_groomed_df,
    'groomed'
)

--- Mode: groomed (Counterpart column: from) ---
記録ミスの可能性がある sampling_id: [7, 51, 72]
入れ替え処理が完了しました。
to 列を name 列の値で補完しました。

[処理結果の確認(代表各1行)]
       sampling_id      name     from        to
903              4     Botan  Komatsu     Botan
1505             7     Botan  Komatsu     Botan
3612            17     Kanna     Kobu     Kanna
5418            27  Tsutsuji    Tsuwa  Tsutsuji
7224            33     Tsuwa  Tsukasa     Tsuwa
7826            35     Tsuwa  Tsukasa     Tsuwa
9632            42    Takana     Tabu    Takana
10234           44    Binega     Hiba    Binega
11739           51   Komatsu   Namiru   Komatsu
12642           56      Yuna    Kanna      Yuna


## `grooming_combination.csv`の作成

In [11]:
strength_matrix = self.build_combined_strength_matrix(
    grooming_df, groomed_df
)

print(strength_matrix)

to        Beni Binega Botan Botaninf  Gure  Hado Hamayu  Hiba HibaInf Hinoki  \
from                                                                           
Beni      <NA>   <NA>     1     <NA>  <NA>  <NA>   <NA>  <NA>    <NA>   <NA>   
Binega    <NA>   <NA>  <NA>     <NA>     3  <NA>   <NA>  <NA>    <NA>   <NA>   
Botan        1   <NA>  <NA>        1  <NA>  <NA>   <NA>  <NA>    <NA>   <NA>   
Gure      <NA>      3  <NA>     <NA>  <NA>  <NA>   <NA>  <NA>    <NA>   <NA>   
Hado      <NA>   <NA>  <NA>     <NA>  <NA>  <NA>   <NA>  <NA>    <NA>   <NA>   
Hamayu    <NA>   <NA>  <NA>     <NA>  <NA>  <NA>   <NA>  <NA>    <NA>   <NA>   
Hiba      <NA>      1  <NA>     <NA>  <NA>  <NA>   <NA>  <NA>       2   <NA>   
Hinoki    <NA>   <NA>  <NA>     <NA>  <NA>  <NA>   <NA>  <NA>    <NA>   <NA>   
Hirugi    <NA>   <NA>  <NA>     <NA>  <NA>  <NA>   <NA>  <NA>    <NA>   <NA>   
Kanna     <NA>   <NA>  <NA>     <NA>  <NA>  <NA>   <NA>  <NA>    <NA>   <NA>   
Kannna    <NA>   <NA>  <NA>     <NA>  <N

In [12]:
# strength_matrixをCSVファイルとして保存
save_path = "../../../data/04_aim_2_data/01_add_label/strength_matrix.csv"

strength_matrix.to_csv(save_path, index=True)

print(f"保存が完了しました: {save_path}")

保存が完了しました: ../../../data/04_aim_2_data/01_add_label/strength_matrix.csv


## rankの考慮

### rank_dfの読み込み

In [13]:
rank_df = load_from_parquet(
    "../../../data/01_relationship_data/rank_df.parquet"
)

--- 復元完了 ---
復元されたDataFrameのshape: (22, 2)


### オスを含めたrank_dictの作成

In [14]:
# 1. 名前から順位を引くための辞書を作成
# rank_df にないオスがいれば、非常に高い順位（例：0）を割り当てる
rank_dict = rank_df.set_index('name')['rank'].to_dict()

# オスのリスト（個別に順位0を割り当て）
males = ['Kobu', 'Nishin', 'Gure']
for male in males:
    rank_dict[male] = 0

毛繕いの順位方向を`rank_direction`列に追加

In [15]:
grooming_df_rank = self.add_rank_direction(
    grooming_df,
    rank_dict
)

# 順位が同じペア（'equal'）はほぼなかったので捨てても良い
drop_cols = ['unknown']  
grooming_df_rank = \
    grooming_df_rank[~grooming_df_rank['rank_direction'].isin(drop_cols)]

順位方向の判定が完了しました。
rank_direction
low_to_high    19565
high_to_low    16014
equal            240
Name: count, dtype: int64


In [16]:
groomed_df_rank = self.add_rank_direction(
    groomed_df,
    rank_dict
)

# 順位が同じペア（'equal'）はほぼなかったので捨てても良い
drop_cols = ['unknown']  
groomed_df_rank = \
    groomed_df_rank[~groomed_df_rank['rank_direction'].isin(drop_cols)]

順位方向の判定が完了しました。
rank_direction
high_to_low    15953
low_to_high    15652
Name: count, dtype: int64


In [17]:
print(grooming_df_rank[['sampling_id', 'name', 'from', 'to', 'rank_direction']].sample(10))

       sampling_id     name     from        to rank_direction
71701          296  Yotsuba  Yotsuba    Teriba    high_to_low
29575          122  Komatsu  Komatsu      Tane    low_to_high
43728          173   Binega   Binega      Gure    low_to_high
72244          299  Yotsuba  Yotsuba    Teriba    high_to_low
2416            10    Botan    Botan  Botaninf    high_to_low
56179          225     Kizu     Kizu    Hirugi    high_to_low
38039          154     Tane     Tane      Beni    high_to_low
79591          323    Tsuwa    Tsuwa      Gure    low_to_high
62114          251     Kizu     Kizu    Hinoki    high_to_low
26551          110     Tane     Tane   Komatsu    high_to_low


## tie_strength

In [18]:
import pandas as pd

# 1. 出現回数を計算（現在の状態）
counts = strength_matrix.stack().value_counts().sort_index()

# 2. DataFrameに変換して「回数」と「数値ごとの和」を計算
df_result = pd.DataFrame(counts)
df_result.columns = ['出現回数']
df_result.index.name = '数値'

# 数値（Index） × 出現回数 で「その数値内の和」を計算
df_result['数値ごとの和'] = df_result.index * df_result['出現回数']

print(df_result)

# 全体の合計も表示
print("-" * 20)
print(f"総計: {df_result['数値ごとの和'].sum()}")

    出現回数 数値ごとの和
数値             
1     27     27
2     17     34
3     16     48
4      7     28
5      4     20
6      3     18
7      1      7
8      2     16
11     1     11
16     1     16
--------------------
総計: 225


### `tie_strength`列を追加

`weak-tie` : 1~2回 (sample:44, total:61)<br>
`midle-tie`: 3~4回 (sample:23, total:76)<br>
`strong-tie`：5~回 (sample:12, total:88)

In [19]:
grooming_df_tie = self.add_tie_strength(
    grooming_df_rank,
    strength_matrix
)

In [20]:
print(grooming_df_tie[['sampling_id', 'from', 'to', 'tie-strength']].sample(10))

       sampling_id     from       to tie-strength
31114          128  Komatsu    Botan   strong-tie
46325          183   Binega  Yotsuba    midle-tie
16229           68     Tabu  TabuInf    midle-tie
59954          242     Kizu   Hirugi   strong-tie
36409          147     Tane  Komatsu   strong-tie
67187          272    Botan     Kobu    midle-tie
58518          234     Hiba   Nishin     weak-tie
31799          130  Komatsu    Botan   strong-tie
29132          120  Komatsu     Tane   strong-tie
48653          197     Yone    Peach    midle-tie


In [21]:
groomed_df_tie = self.add_tie_strength(
    groomed_df_rank,
    strength_matrix
)

## centrality

### centralityのdictを作成

In [22]:
centrality_df = load_from_parquet(
    "../../../data/01_relationship_data/centrality_df.parquet"
)

--- 復元完了 ---
復元されたDataFrameのshape: (34, 2)


In [23]:
# 名前から中心性を引くための辞書を作成
centrality_dict = centrality_df.set_index('name')['centrality'].to_dict()
print(centrality_dict)

{'Komatsu': 51.0, 'Yotsuba': 46.0, 'Botan': 39.0, 'Tane': 30.0, 'Kizu': 21.0, 'Beni': 20.0, 'Yone': 18.0, 'Hirugi': 16.0, 'Tabu': 15.0, 'Binega': 14.0, 'Mikan': 14.0, 'Teriba': 13.0, 'Kobu': 11.0, 'Gure': 11.0, 'Peach': 11.0, 'Takana': 11.0, 'Tsuwa': 10.0, 'Yuna': 10.0, 'Namiru': 10.0, 'Nishin': 9.0, 'Hiba': 9.0, 'TabuInf': 8.0, 'Kannna': 7.0, 'Taraba': 7.0, 'Kanna': 7.0, 'Tsutsuji': 6.0, 'hamayu': 6.0, 'Tsukasa': 5.0, 'Hamayu': 5.0, 'Hinoki': 4.0, 'HibaInf': 2.0, 'Hado': 2.0, 'Botaninf': 1.0, 'YunaInf': 1.0}


### 中心性の方向性を付与（`centrality_direction`列）

In [24]:
grooming_df_centrality = self.add_centrality_direction(
    grooming_df_tie,
    centrality_dict
)

中心性方向の判定が完了しました。
centrality_direction
high_to_low    23478
low_to_high    11739
equal            602
Name: count, dtype: int64


In [25]:
groomed_df_centrality = self.add_centrality_direction(
    groomed_df_tie,
    centrality_dict
)

中心性方向の判定が完了しました。
centrality_direction
low_to_high    18060
high_to_low    12341
equal           1204
Name: count, dtype: int64


## 最終的なdfの保存

In [26]:
save_to_parquet(
    grooming_df_centrality,
    "../../../data/04_aim_2_data/01_add_label/labeled_grooming_df.parquet"
)

--- 保存完了 ---
'../../../data/04_aim_2_data/01_add_label/labeled_grooming_df.parquet' として保存されました。ファイルサイズ: 1336.68 KB


In [27]:
save_to_parquet(
    groomed_df_centrality,
    "../../../data/04_aim_2_data/01_add_label/labeled_groomed_df.parquet"
)

--- 保存完了 ---
'../../../data/04_aim_2_data/01_add_label/labeled_groomed_df.parquet' として保存されました。ファイルサイズ: 1133.62 KB
